In [46]:
import sys
import os
import numpy as np
import pandas as pd 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

In [47]:
df=pd.read_csv('fer2013.csv')

In [48]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None


In [49]:
print(df["Usage"].value_counts())

Training       28709
PrivateTest     3589
PublicTest      3589
Name: Usage, dtype: int64


In [50]:
print(df.head())

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


In [51]:
X_train,train_y,X_test,test_y=[],[],[],[]

for index,row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            X_train.append(np.array(val, 'float32'))
            train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            X_test.append(np.array(val, 'float32'))
            test_y.append(row['emotion'])
    except:
        print(f"error occured at index: {index} and row:{row}")
            

In [52]:
print(f"X_train sample data : {X_train[0:2]}")
print(f"train_y sample data : {train_y[0:2]}")
print(f"X_test sample data : {X_test[0:2]}")
print(f"test_y sample data : {test_y[0:2]}")

X_train sample data : [array([ 70.,  80.,  82., ..., 106., 109.,  82.], dtype=float32), array([151., 150., 147., ..., 193., 183., 184.], dtype=float32)]
train_y sample data : [0, 0]
X_test sample data : [array([254., 254., 254., ...,  42., 129., 180.], dtype=float32), array([156., 184., 198., ..., 172., 167., 161.], dtype=float32)]
test_y sample data : [0, 1]


In [53]:
X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

In [54]:
X_train-=np.mean(X_train,axis=0)
X_train/=np.std(X_train, axis=0)

X_test-=np.mean(X_test,axis=0)
X_test/=np.std(X_test, axis=0)

In [55]:
num_features = 64
num_labels = 7
batch_size = 64
epochs=30
width, height = 48,48


In [56]:
X_train = X_train.reshape(X_train.shape[0],width,height,1)

X_test = X_test.reshape(X_test.shape[0],width,height,1)

train_y = np_utils.to_categorical(train_y, num_classes=num_labels)
test_y = np_utils.to_categorical(test_y, num_classes=num_labels)

In [57]:
model=Sequential()

model.add(Conv2D(num_features, kernel_size=(3,3), activation='relu', input_shape = (X_train.shape[1:])))
model.add(Conv2D(num_features, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(num_features, (3,3),activation='relu'))
model.add(Conv2D(num_features,(3,3) ,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, (3,3),activation='relu'))
model.add(Conv2D(2*num_features,(3,3) ,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

In [58]:
model.add(Flatten())


In [59]:
model.add(Dense(2*2*2*2*num_features,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2*2*2*2*num_features,activation='relu'))
model.add(Dropout(0.2))

In [60]:
model.add(Dense(num_labels, activation='softmax'))


In [61]:
model.compile(loss=categorical_crossentropy, optimizer = Adam(), metrics=['accuracy'])

In [62]:
model.fit(X_train, train_y , batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, test_y), shuffle=True)


Train on 28709 samples, validate on 3589 samples
Epoch 1/30
28709/28709 [==============================] - 336s 12ms/step - loss: 1.7293 - accuracy: 0.2930 - val_loss: 1.7215 - val_accuracy: 0.3410
Epoch 2/30
28709/28709 [==============================] - 334s 12ms/step - loss: 1.5150 - accuracy: 0.4007 - val_loss: 1.3906 - val_accuracy: 0.4631
Epoch 3/30
28709/28709 [==============================] - 335s 12ms/step - loss: 1.4021 - accuracy: 0.4553 - val_loss: 1.3245 - val_accuracy: 0.4907
Epoch 4/30
28709/28709 [==============================] - 333s 12ms/step - loss: 1.3351 - accuracy: 0.4828 - val_loss: 1.2694 - val_accuracy: 0.5099
Epoch 5/30
28709/28709 [==============================] - 334s 12ms/step - loss: 1.2903 - accuracy: 0.5030 - val_loss: 1.2589 - val_accuracy: 0.5149
Epoch 6/30
28709/28709 [==============================] - 351s 12ms/step - loss: 1.2515 - accuracy: 0.5192 - val_loss: 1.2221 - val_accuracy: 0.5397
Epoch 7/30
28709/28709 [==============================] 

In [64]:
fer_json=model.to_json()
with open("fer.json","w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")